In [ ]:
%pip install instructor

In [67]:
import os
from dotenv import load_dotenv

load_dotenv()
OPEN_API_KEY = os.getenv("OPEN_API_KEY")

In [68]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import instructor

class SubTask(BaseModel):
    action: str
    assignee: str
    requires_tool : bool = Field(..., description="Does this require the use of a specific tool ?")

class Task(BaseModel):
    sub_task:List[SubTask] = Field(..., description="List of sub tasks")

class ParsedPlan(BaseModel):
    tasks : List[Task]
messages= [
        dict(
            role="system", 
            content="You help parse markdown into a structured format."),
        dict(
            role="user",
            content=f"Here is the context about the plan including the available tools :\n{context}\n\nThe plan: {plan}")
    ]


In [69]:
## using openai 
openai_client = OpenAI( api_key=OPEN_API_KEY)   

def get_plan_using_openai(plan:str, context:str): 
    return openai_client.beta.chat.completions.parse(
            response_format=ParsedPlan,
            model="gpt-4o",
            messages= messages
        )

In [70]:
## using instructor
client = instructor.from_openai(OpenAI(api_key=OPEN_API_KEY))
def get_plan_using_instructor(plan:str, context:str): 
    return client.chat.completions.create(
            response_model=ParsedPlan,
            model="gpt-4o",
            messages= messages
        )

In [71]:
plan = """## Step 1: Prepare for Workspace Creation\n\n
### 1.1 Verify Permissions\n- ensure you have the necessary permissions to create resources in Azure subscription. \n
Typically, you need to be an Owner or Contributor on the subscription. \n\n
### 1.2 Gather Prerequisites\n - Subscription ID\n- Resource Group (create a new one if necessary)\n- Region where the workspace will be created\n- Storage account details (if you plan to use an existing one)\n\n
## Step 2: Access Azure Portal\n\n
### 2.1 Log In\n- Navigate to the [Azure Portal](https://portal.azure.come).\n- Log in with your Azure Credentials.\n\n
## Step 3: Create the Synapse Workspace\n\n
### 3.1 Navigate to Synapse Service\n- In the Azure portal, click on 'Create a resource'.\n- Search for 'Azure Synapse Analytics' and select it.\n\n
### 3.2 Start the creation Process\n- Click on the 'Create' button to start the workspace creation process.
"""
context = "I have access to Azure portal and Azure Synapse workspace"


In [76]:
result = get_plan_using_openai(plan=plan, context=context)
#print(result.choices[0].message.parsed)
for t in result.choices[0].message.parsed.tasks:
    print(t)
    print('\n')

sub_task=[SubTask(action='Ensure you have the necessary permissions to create resources in Azure subscription. Typically, you need to be an Owner or Contributor on the subscription.', assignee='', requires_tool=False), SubTask(action='Gather the following prerequisites: Subscription ID, Resource Group (create a new one if necessary), Region where the workspace will be created, Storage account details (if using an existing one)', assignee='', requires_tool=False)]


sub_task=[SubTask(action='Navigate to the Azure Portal.', assignee='', requires_tool=True), SubTask(action='Log in with your Azure Credentials.', assignee='', requires_tool=True)]


sub_task=[SubTask(action="In the Azure portal, click on 'Create a resource'. Search for 'Azure Synapse Analytics' and select it.", assignee='', requires_tool=True), SubTask(action="Click on the 'Create' button to start the workspace creation process.", assignee='', requires_tool=True)]




In [73]:
result = get_plan_using_instructor(plan=plan, context=context)
for t in result.tasks:
    print(t)
    print('\n')

sub_task=[SubTask(action='Ensure you have the necessary permissions to create resources in Azure subscription.', assignee='Owner or Contributor on the subscription', requires_tool=False)]


sub_task=[SubTask(action='Gather Subscription ID, Resource Group, Region, and Storage account details (if using an existing one).', assignee='Individual creating the workspace', requires_tool=False)]


sub_task=[SubTask(action='Navigate to the Azure Portal and log in with your Azure Credentials.', assignee='Individual accessing Azure Portal', requires_tool=False)]


sub_task=[SubTask(action="In the Azure portal, click on 'Create a resource' and search for 'Azure Synapse Analytics'.", assignee='Individual creating Synapse Workspace', requires_tool=False)]


sub_task=[SubTask(action="Click on the 'Create' button to start the workspace creation process.", assignee='Individual creating Synapse Workspace', requires_tool=False)]


